# Project 3: Understanding User Behavior

Authors: Leyla Greengard, Tony Hwang, Ryan Orton, Jun Cao

**Objective**: Our intent is to understand how behavior of users playing the video game. Most importantly, we are trying to understand the relationship between joining guilds and purchasing game materials - swords, armor, helmets. 

**Outcome**: We found three queries to run on our data, analysis of these queries is near bottom of document (note that "fake data" is used): 

* Note: our data was generated by our scripts, and is an example to show executive management how similar analytics can help drive game design. 

#### Events description:
"Buy an item" 
Players can buy one or more of several items. All items' different characteristics and costs are loaded in HDFS, in the file item_meta. A description of the file follows. 
"Join a guild"
Players can also join one or more guilds. Joining guilds does not cost anything, but we are interested in the events because they may encourage players to buy items later on. The guilds' characteristics are also saved in HDFS. The name of the file is guild_meta

#### Our approach: 
    Our approach is focused on demonstrating both the steps necessary to create, and the power of creating an events based data pipeline. Step by step instructions of how we created, and ran the pipeline are below. We run in several shell terminals, indicated below.:

**Step 1) Start-up Docker, check status, confirm networks ready to run**
- In TERMINAL 1
- spinning up containers
- CMD: docker-compose up -d
- CMD: docker-compose ps
- CMD: docker network ls (show network)
- CMD: docker network prune (prune network)

Output: docker containers up and running.

**Step 2) Create topic in Kafka to catch events**
- In TERMINAL 2
- CMD: docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Output: Created topic events.


**Step 3) Run flask app (game_api.py) to respond to api requests**
- TERMINAL 2
- CMD: docker-compose exec mids env FLASK_APP=/w205/w205-project3/game_api-TH.py flask run --host 0.0.0.0

Output: Flask app up and running, pending requests from API.  

**Step 4) Write two events and Infer schema**
- TERMINAL 3
- docker-compose exec mids ab -n 1 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword1
- docker-compose exec mids ab -n 1 -H "Host: user1.comcast.com" http://localhost:5000/join_a_guild1
- CMD: docker-compose exec spark spark-submit /w205/w205-project3/write_hive_table.py

In order to infer the schemas we start with one purchase and join_guild event, and utilize HIVE in spark in order to create the two files in HDFS, and manually create metadata tables.
Result: Schemas have been inferred

**Step 5) get ready to filter events and write to 2 parquet files:  purchases and join_guild**
- TERMINAL 4
- CMD: docker-compose exec spark spark-submit /w205/w205-project3/write_events_stream.py

Result: At this point we are ready to process continuous events

**Step 6) at this point we started to generate data in order to demonstrate our process**
- TERMINAL 3
- CMD: bash events.sh
    
events.sh is a bash shell that continously creates purchase and join_guild events

Result: demo data created and written to HDFS

**Step 7) We can now query our files using Presto**
- TERMINAL 5
- CMD: docker-compose exec presto presto --server presto:8080 --catalog hive --schema default

Result: Presto is set up, Presto prompt appears 


**Step 8) Query events files**
- TERMINAL 5
Presto commands:
    
**show tables**

presto:default> show tables;
    
**check purchases file's contents** 

presto:default> describe purchases;
    
presto:default> select * from purchases;

presto:default> select count(*) from purchases;
    
**check join_guild file's contents** 

presto:default> describe purchases;

presto:default> select * from purchases;

presto:default> select count(*) from purchases;

**Shut down**

CMD - docker-compose down

Result: all HDFS files are deleted

### Business Questions

Our current setup allows us to answers many questions such as: 
    
1) Do users buy the more of the more expensive sword or cheaper sword? What about helmet and armor?
    
2) Does joining a second guild lead to additional purchases?
  - Query per user, total count of purchases before going second guild event and after
    
3) Do users buy the more of the more expensive sword or cheaper sword? What about helmet and armor?  - Assume: most expensive to least: armor - helmet - sword
  - Query: count number of armor, helmet, sword purchases first half of transactions vs. second half of transactions, by user
    
4) which type of equipment is most popular and how much have we earned from selling equipment?

5) which color is most popular and how much have we earned by color?

6) what has been our total the revenue?

7) How many people made a purchase after joining a guild? and how much did they spend?

In [ ]:
**Queries to answer business questions**

1. `select description, count(*) FROM purchases group by description;`

5. select 

### Answers and intepretation after running queries to business questions

1. For swords, slightly more of the more expensive sword was bought. From a business perspective, the comparitive pricing of swords is probably ok. For armor, more of the more expensive armor was bought. One note, the more expensive armor is 50% more than the cheaper armor. We may be able to raise the prie of the more expensive armor from $6 to $7, possibly and still be ok due to the demand. For the helmet, much more of the cheaper helmet was bought than the more expensive helmet, even though the price difference is just $3 compared to $4. Look into why, maybe the cheaper helmet is almost as good as the more expensive helmet? Maybe the solution would be to increase the ratings of the more expensive helmet so it becomes more desirable.
1. 

### Files descriptions

The system comprises 4 files in \tmp\ on HDFS. Their description follows.

**File name: item-meta**
    
Description: static file of items that are for sale
    
Format

    Field      | type   | description
    --------------------------------------------------------------
    item_id    | string | 4 character unique identifier 
    item_name  | string | up to 25 characters
    item_type  | string | up to 25 characters: sword, armour, etc
    item_color | string | up to 15 characters
    item_cost  | int    | cost in US dollars
    

**File name: guild-meta**
    
Description: static file of available guilds

Format

    Field        | type   | description
    -------------------------------------------------------------
    guild_id     | string | 4 character unique identifier 
    guild_name   | string | up to 25 characters
    geo location | string | location of the guild
    guild size   | string | small, regular, large,...


**File name: Purchases**
    
Description: all purchases. This file is updated every time an item is purchased

Format

    Field       | type   | description
    ------------------------------------------------------------
    Accept      | string | "*/*"
    Host        | string | user and service provider
    user agent  | string | "ApacheBench/2.3"
    event type  | string | purchase
    description | string | item name 
    timestamp   | string | approximate date and time of purchase 

**File name: Join_guilds**
    
Description: all joinings of a guild. This file is updated every time a guild is joined
    
Format

    Field       | type   | description 
    ------------------------------------------------------------
    Accept      | string | "*/*"
    Host        | string | user and service provider
    user agent  | string | "ApacheBench/2.3"
    event type  | string | join guild
    description | string | guild name 
    timestamp   | string | approximate date and time of purchase 
    
note that the timestamp is the date/time at which the record entered the queue in kafka, which is slightly later that the actual event. However, this is sufficient for our purposes.  